## Invoke SageMaker Enpoint from outside of AWS environment using SageMaker SDK

Model used: XGBoost Bike Rental Prediction Trained in the XGBoost Lectures  
  
This example uses the IAM user: ml_user_predict. The user was setup in the housekeeping lecture of the course.  

Refer to the lecture: Configure IAM Users, Setup Command Line Interface (CLI)

Ensure xgboost-biketrain-v1 Endpoint is deployed before running this example  
  
To create an endpoint using SageMaker Console:  
1. Select "Models" under "Inference" in navigation pane
2. Search for model using this prefix: xgboost-biketrain-v1
3. Select the latest model and choose create endpoint
4. Specify endpoint name as: xgboost-biketrain-v1
5. Create a new endpoint configuration
6. Create a new endpoint
7. After this lab is completed, delete the endpoint to avoid unnecessary charges

In [1]:
# Install SageMaker 2.x version.
#!pip install --upgrade sagemaker

In [2]:
import boto3
import sagemaker
import numpy as np

In [3]:
# Establish a session with AWS
# Specify credentials and region to be used for this session.
# We will use a ml_user_predict credentials that has limited privileges

boto_session = boto3.Session(profile_name='ml_user_predict', region_name='us-east-1')

#boto_session = boto3.Session()

In [4]:
sess = sagemaker.Session(boto_session=boto_session)

In [5]:
# Create a predictor and point to an existing endpoint

# Get Predictor using SageMaker SDK
# Specify Your Endpoint Name
#endpoint_name = 'xgboost-biketrain-v1'
endpoint_name = 'sagemaker-xgboost-2022-11-28-15-44-11-695'
#endpoint_name = 'linear-learner-2022-11-15-14-57-02-049'

predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name,
                                                 sagemaker_session=sess)

In [6]:
# test model on libsvm file
from sagemaker.serializers import LibSVMSerializer
predictor.serializer = LibSVMSerializer

In [7]:
# predict on libsvm file
with open('./data/validation', "r") as f:
    payload = f.read()

In [8]:
runtime_client = sess.sagemaker_runtime_client
response = runtime_client.invoke_endpoint(
    EndpointName=predictor.endpoint_name, 
    ContentType="text/libsvm", Body=payload
)

In [9]:
result = response["Body"].read().decode("ascii")
predicted_values = np.fromstring(result[1:-2], dtype=float, sep=',')
print("Predicted values are {}.".format(predicted_values[:10]))

Predicted values are [ 6767.17675781 33017.98046875 49357.37109375  8696.41796875
  4399.16503906 11170.62890625 55041.44140625  9964.31152344
  4525.75292969  5143.10693359].


In [10]:
predicted_values.shape

(105393,)

In [11]:
# Delete the end-point
#predictor.delete_model
#predictor.delete_endpoint()